In [2]:
import os
import io
import cv2
import fitz
import PyPDF2
import camelot
import pytesseract
import numpy as np
import pandas as pd
from wand.image import Image
import matplotlib.pyplot as plt
from pdf2image import convert_from_path

In [6]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
os.environ['PATH'] += r';C:\Program Files\Tesseract-OCR'

In [13]:
# In case of .pdf file
pages = []
file = '6.pdf'
doc = fitz.open(file)
for n in range(doc.page_count):
    page = doc.load_page(n)
    pix = page.get_pixmap()
    image = np.frombuffer(pix.samples,
                          dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
    image = np.ascontiguousarray(image[..., [2, 1, 0]])
    pages.append(image)

# image = pages[3]
image = cv2.imread('high_res.jpg')
# Отображение изображения с выделенными ячейками
# cv2.imshow('Image with Cells', image)
# cv2.imwrite('image.jpg', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Для выделения таблицы из корпуса текста попробуем использовать cv2 с такими настройками

In [14]:
# Преобразование изображения в оттенки серого
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Применение порогового преобразования для бинаризации изображения
ret, threshold_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Применение морфологического преобразования для удаления шума и замыкания линий
kernel = np.ones((5, 5), np.uint8)
morph_image = cv2.morphologyEx(threshold_image, cv2.MORPH_CLOSE, kernel)

# Нахождение контуров на изображении
contours, hierarchy = cv2.findContours(morph_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Создание списка координат ограничивающих рамок ячеек
cell_boxes = []

# Фильтрация контуров и определение ограничивающих рамок ячеек

height, width, _ = image.shape

parts_image = []

for contour in contours:
    # Определение площади контура
    area = cv2.contourArea(contour)

    x, y, w, h = cv2.boundingRect(contour)

    # cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Фильтрация контуров по площади
    if area > 3000:
        # Определение ограничивающей рамки контура
        x, y, w, h = cv2.boundingRect(contour)

        # Добавление ограничивающей рамки в список
        cell_boxes.append((x, y, x + w, y + h))

        crop_image = image[y:y + h, x:x + w]

        parts_image.append(crop_image)

# Вывод координат ограничивающих рамок ячеек
for i, box in enumerate(cell_boxes):
    x1, y1, x2, y2 = box
    # cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    # cv2.imshow('Image with Cells', parts_image[i])
    # cv2.imwrite('parts_image.jpg', parts_image[i])
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

# Отображение изображения с выделенными ячейками
cv2.imshow('Image with Cells', morph_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
# Загрузка изображения
image = parts_image[0].copy()

# Преобразование изображения в оттенки серого
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Отображение изображения с ограничивающими рамками символов
# cv2.imshow('Image with Character Boxes', gray_image)
# cv2.imwrite('gray_image.jpg', gray_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

_, threshold_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Применение морфологического преобразования для удаления шума и замыкания линий
kernel = np.ones((5, 5), np.uint8)
morph_image = cv2.morphologyEx(threshold_image, cv2.MORPH_CLOSE, kernel)

# Отображение изображения с ограничивающими рамками символов
# cv2.imshow('Image with Character Boxes', morph_image)
# cv2.imwrite('morph_image.jpg', morph_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [16]:
# Загрузка изображения
image = parts_image[0]
cv2.imwrite('image.jpg', image)

# Преобразование изображения в оттенки серого
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

_, threshold_image = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY)

# Получение ограничивающих рамок символов
boxes = pytesseract.image_to_boxes(threshold_image, lang='rus')

# Разделение строки на список строк
box_lines = boxes.strip().split('\n')

# Преобразование каждой строки в список
boxes_list = [line.split() for line in box_lines]

# print(boxes_list)

# Так как cv2 и tesseract используют разные системы координат
height, width = gray_image.shape

# Список границ таблицы
borders = []

epsilon = 10

# Вывод ограничивающих рамок символов
for box in boxes_list:
    if box[0] == '~' and ((abs(int(box[1]) - int(box[3])) <= epsilon) or (abs(int(box[2]) - int(box[4])) <= epsilon)):
        x, y, w, h = int(box[1]), int(box[2]), int(box[3]), int(box[4])
        borders.append([x, y, w, h])
        cv2.rectangle(image, (x, height - y), (w, height - h), (128, 0, 128), 1)

# Отображение изображения с ограничивающими рамками символов
cv2.imshow('Image with Character Boxes', image)
cv2.imwrite('box_image.jpg', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
image.shape

(445, 806, 3)

Попробуем найти координаты всех узлов таблицы, по этим узлам восстановим прямоугольники ячеек

In [241]:
import itertools

# Находим все вертикальные и горизонтальные линии каркаса таблицы
vertical_lines = []
horizontal_lines = []

# Параметры допуска при котором считаем что линии ровные и пересекаются
epsilon = 10

for x1, y1, x2, y2 in borders:
    if abs(x1 - x2) < epsilon:  # Вертикальная линия
        vertical_lines.append((int((x1 + x2) / 2), y1, int((x1 + x2) / 2), y2))
    elif abs(y1 - y2) < epsilon:  # Горизонтальная линия
        horizontal_lines.append((x1, int((y1 + y2) / 2), x2, int((y1 + y2) / 2)))

# Находим координаты узлов таблицы
table_nodes = []

for v_line, h_line in itertools.product(vertical_lines, horizontal_lines):
    v_x1, v_y1, v_x2, v_y2 = v_line
    h_x1, h_y1, h_x2, h_y2 = h_line

    # Проверяем, пересекаются ли прямые или заканчивается ли одна из них в окрестности
    if ((h_x1 - epsilon <= v_x1 <= h_x2 + epsilon or h_x1 - epsilon <= v_x2 <= h_x2 + epsilon) and
        (v_y1 - epsilon <= h_y1 <= v_y2 + epsilon or v_y1 - epsilon <= h_y2 <= v_y2 + epsilon)) or (
            abs(h_x2 - v_x1) <= epsilon and v_y1 <= h_y1 <= v_y2) or (
            abs(h_x2 - v_x2) <= epsilon and v_y1 <= h_y1 <= v_y2) or (
            abs(h_x1 - v_x1) <= epsilon and v_y1 <= h_y1 <= v_y2) or (
            abs(h_x1 - v_x2) <= epsilon and v_y1 <= h_y1 <= v_y2):
        # Добавляем координаты узла в список
        table_nodes.append((v_x1, h_y1))

    if v_x1 <= epsilon or v_y1 <= epsilon or abs(v_x1 - width) <= epsilon or abs(v_y1 - height) <= epsilon:
        table_nodes.append((v_x1, v_y1))

    if v_x2 <= epsilon or v_y2 <= epsilon or abs(v_x2 - width) <= epsilon or abs(v_y2 - height) <= epsilon:
        table_nodes.append((v_x2, v_y2))

    if h_x1 <= epsilon or h_y1 <= epsilon or abs(h_x1 - width) <= epsilon or abs(h_y1 - height) <= epsilon:
        table_nodes.append((h_x1, h_y1))

    if h_x2 <= epsilon or h_y2 <= epsilon or abs(h_x2 - width) <= epsilon or abs(h_y2 - height) <= epsilon:
        table_nodes.append((h_x2, h_y2))

    if v_y1 <= epsilon or v_y2 <= epsilon:
        table_nodes.append((0, 0))
        table_nodes.append((width, 0))

In [242]:
# Создаем копию списка table_nodes для модификации
modified_table_nodes = table_nodes.copy()

# Проходим по каждой точке в списке table_nodes
for i, node in enumerate(table_nodes):
    x, y = node
    count = 1
    avg_x, avg_y = x, y

    # Ищем точки в окрестности epsilon
    for j, other_node in enumerate(table_nodes):
        if i != j:  # Игнорируем текущую точку
            other_x, other_y = other_node

            # Проверяем, находится ли другая точка в окрестности epsilon
            if abs(other_x - x) <= epsilon and abs(other_y - y) <= epsilon:
                avg_x += other_x
                avg_y += other_y
                count += 1

    # Если найдены точки в окрестности, заменяем текущую точку на среднее значение
    if count > 1:
        avg_x /= count
        avg_y /= count
        modified_table_nodes[i] = (int(avg_x), int(avg_y))

f = True

while f:
    for i, node in enumerate(modified_table_nodes):
        if i == len(modified_table_nodes) - 1:
            f = False
        else:
            x, y = node
            count = 1
            indices = []
            avg_x, avg_y = x, y

            # Ищем точки в окрестности epsilon
            for j, other_node in enumerate(modified_table_nodes):
                if i != j:  # Игнорируем текущую точку
                    other_x, other_y = other_node

                    # Проверяем, находится ли другая точка в окрестности epsilon
                    if abs(other_x - x) <= epsilon and abs(other_y - y) <= epsilon:
                        avg_x += other_x
                        avg_y += other_y
                        count += 1
                        indices.append(j)

            # Если найдены точки в окрестности, заменяем текущую точку на среднее значение
            if count > 1:
                avg_x /= count
                avg_y /= count
                modified_table_nodes[i] = (int(avg_x), int(avg_y))

            for item in sorted(indices, reverse=True):
                modified_table_nodes.pop(item)

# Отсортируем узлы по оси x и y
epsilon = 5
nodes_sorted_x = sorted(modified_table_nodes, key=lambda x: x[0])

for i in range(len(nodes_sorted_x)-1):
    if abs(nodes_sorted_x[i][0] - nodes_sorted_x[i+1][0]) <= epsilon:
        nodes_sorted_x[i+1] = (nodes_sorted_x[i][0], nodes_sorted_x[i+1][1])

nodes_sorted_y = sorted(nodes_sorted_x, key=lambda x: x[1])

for i in range(len(nodes_sorted_y)-1):
    if abs(nodes_sorted_y[i][1] - nodes_sorted_y[i+1][1]) <= epsilon:
        nodes_sorted_y[i+1] = (nodes_sorted_y[i+1][0], nodes_sorted_y[i][1])

nodes_sorted_xy = sorted(nodes_sorted_y, key=lambda x: (-x[1], x[0]))

# Подсчет уникальных значений координат x и y
unique_x = set()
unique_y = set()

# Определение уникальных координат x и y
for x, y in nodes_sorted_xy:
    unique_x.add(x)
    unique_y.add(y)
    
# Определение числа строк и столбцов таблицы
num_rows = len(unique_y) - 1
num_columns = len(unique_x) - 1
print(num_rows, num_columns)  

while True:
    count_x = {}
    count_y = {}
    for x, y in nodes_sorted_xy:
        count_x[x] = count_x.get(x, 0) + 1
        count_y[y] = count_y.get(y, 0) + 1

    filtered_points = [(x, y) for x, y in nodes_sorted_xy if count_x[x] > 1 and count_y[y] > 1]
    tresh_points = [(x, y) for x, y in nodes_sorted_xy if count_x[x] <= 1 or count_y[y] <= 1]

    if len(filtered_points) == len(nodes_sorted_xy):
        break
    else:
        nodes_sorted_xy = filtered_points

# # Выводим координаты узлов на изображении
# for node in nodes_sorted_xy:
#     x, y = node
#     cv2.rectangle(image, (x - 3, height - y - 3), (x + 3, height - y + 3), (0, 0, 255), 1)

# # Отображение изображения
# cv2.imshow('Image', image)
# cv2.imwrite('nodes_image.jpg', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

10 10


In [243]:
# Создадим список прямоугольников
rectangles = []

# Пройдем по узлам и создадим прямоугольники
for i in range(len(nodes_sorted_xy) - 1):
    current_node = nodes_sorted_xy[i]
    if nodes_sorted_xy[i + 1][1] == current_node[1]:
        next_x_node = nodes_sorted_xy[i + 1]
    else:
        continue
    # Найдем узлы следующие по оси y для текущих узлов
    next_y_nodes = [node for node in nodes_sorted_xy if abs(node[0] - current_node[0]) <= epsilon and node[1] < current_node[1]]

    # print(current_node, next_x_node, next_y_nodes)
    flag = True

    for next_y_node in next_y_nodes:
        opposite_node = (next_x_node[0], next_y_node[1])
        if flag:
            for node in nodes_sorted_xy:
                if abs(node[0] - opposite_node[0]) <= epsilon and abs(node[1] - opposite_node[1]) <= epsilon:
                    # Построим прямоугольник по найденным узлам
                    rectangle = ((current_node[0], current_node[1]), (opposite_node[0], opposite_node[1]))
                    rectangles.append(rectangle)
                    # cv2.rectangle(image, (current_node[0] + 2, height - current_node[1] + 2), (opposite_node[0] - 2, height - opposite_node[1] - 2), (128, 0, 128), 1)
                    # cv2.imshow('Image', image)
                    # cv2.waitKey(0)
                    # cv2.destroyAllWindows()
                    flag = False
                    break

# Выведем список прямоугольников
# print(rectangles)

# # Отображение изображения
# cv2.imshow('Image', image)
# cv2.imwrite('rectangle_image.jpg', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [246]:
rectangles = sorted(rectangles, key=lambda x: (- x[0][1], x[0][0]))
print(rectangles)

# Создание пустого DataFrame для хранения данных таблицы
table_data = [['' for _ in range(num_columns)] for _ in range(num_rows)]

# Обход прямоугольников ячеек таблицы
for i, rectangle in enumerate(rectangles):
    # Извлечение координат углов прямоугольника
    x1, y1 = rectangle[0]
    x2, y2 = rectangle[1]

    # Обрезание изображения по границам ячейки
    cell_image = image[max(0, (height - y1) - 2):min(height, (height - y2) + 2),
                 max(0, x1 - 2):min(width, x2 + 2)]

    # Russian
    custom_config = r'--oem 1 --psm 1 -l rus'

    if cell_image.shape[0] > 10 and cell_image.shape[1] > 10:
        gray_image = cv2.cvtColor(cell_image, cv2.COLOR_BGR2GRAY)
        _, threshold_image = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY)
        # # Отображение изображения
        # cv2.imshow('Image', threshold_image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

        # Распознавание текста с помощью Tesseract
        text = pytesseract.image_to_string(gray_image, config=custom_config).replace('\n', ' ')

        print(text)

        # Определение положения ячейки в таблице
        row = i // num_columns
        column = i % num_columns

        # Добавление данных ячейки
        if row < num_rows and column < num_columns:
            table_data[row][column] = text

# Вывод полученной таблицы
df = pd.DataFrame(table_data)
# Сохранение DataFrame в формате CSV
df.to_csv('table.csv', index=False, header=False)
# Записать данные в файл XLSX
df.to_excel('file.xlsx', index=False, encoding='utf-8')

[((79, 441), (243, 283)), ((243, 441), (460, 416)), ((460, 441), (539, 312)), ((243, 416), (315, 312)), ((315, 416), (389, 312)), ((389, 416), (460, 312)), ((539, 400), (744, 376)), ((744, 400), (804, 312)), ((539, 376), (609, 312)), ((609, 376), (682, 312)), ((682, 376), (744, 312)), ((243, 312), (315, 283)), ((315, 312), (389, 283)), ((389, 312), (460, 283)), ((460, 312), (539, 283)), ((539, 312), (609, 283)), ((609, 312), (682, 283)), ((682, 312), (744, 283)), ((744, 312), (804, 283)), ((0, 283), (79, 232)), ((79, 283), (243, 232)), ((243, 283), (315, 232)), ((315, 283), (389, 232)), ((389, 283), (460, 232)), ((460, 283), (539, 232)), ((539, 283), (609, 232)), ((609, 283), (682, 232)), ((682, 283), (744, 232)), ((744, 283), (804, 232)), ((0, 232), (79, 164)), ((79, 232), (243, 164)), ((243, 232), (315, 164)), ((315, 232), (389, 164)), ((389, 232), (460, 164)), ((460, 232), (539, 164)), ((539, 232), (609, 164)), ((609, 232), (682, 164)), ((682, 232), (744, 164)), ((744, 232), (804, 1

c:\Users\dsash\anaconda3\Lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


В качестве альтаернативы tesseract можено использовать easyOCR

In [41]:
!pip install easyocr

  Using cached easyocr-1.7.0-py3-none-any.whl (2.9 MB)
  Using cached python_bidi-0.4.2-py2.py3-none-any.whl (30 kB)
  Using cached ninja-1.11.1-py2.py3-none-win_amd64.whl (313 kB)
     -------------------------------------- 94.2/94.2 kB 449.2 kB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 787.6 kB/s eta 0:00:00
  Using cached opencv_python_headless-4.8.0.74-cp37-abi3-win_amd64.whl (38.0 MB)
     -------------------------------------- 162.6/162.6 MB 1.7 MB/s eta 0:00:00


In [42]:
import easyocr

In [45]:
# Создание экземпляра класса EasyOCR
reader = easyocr.Reader(['en', 'ru'], gpu=False)  # указание языка (например, 'en' для английского)

# Загрузка изображения и распознавание текста
image_path = 'high_res.jpg'
result = reader.readtext(image_path)

# Вывод результатов
for detection in result:
    text = detection[1]
    print(text) 

Using CPU. Note: This module is much faster with a GPU.


2.18.
Свариваемость стали гарантируется изготовителем.
По требованию потребителя углеродный эквивалент (Сэ)
должен быть для стали СЗ9О и СЗ9ОК
не более 0,49 %
стали
С440
не более 0,51 %
2.19.
Механические свойства при растяжении
ударная ВЯзкость
также
условия испытаний
на изгиб должны соответствовать для фасонного проката
требованиям табл. 3 , листового и широко
полосного универсального
табл: 4.
Ta 6 л и ц a
Механическне свойства фасонного проката
Механические
характеристики
Изгиб до
Уларная вязкость KCU; Дж/см-
параллель
(кгС
м/см-)
Наимено-
Предел
Временное"
Относи-
HОСТИ
Толшина полки,
текучести
сопротив -
тельное
при
температуре
после
вание
сторон (a
ми
Н мм'
ление
удлине -
механи
стали
толшина
(кгс мм-)
Н мм-
ние
б5ж %o
образца
_40
ческого
(кгс / мм?)
диаметр
старе-
Ня
оправки )
Hе ченее
не менее
С235
От
4 до 20 включ.
235(24)
360(37)
26
d = d
Св: 20
40
225(23)
360(37)
d = 2a
C245
От
4 до 20 включ
245(25)
370(38)
d = d
29(3)
Св. 20
235(24
370(38)
?5
d = 2a
29(3)
25
30
235(24)
370(

Для readable pdf файлов можно использовать готовое решение tabula или camelot

In [37]:
def process_tables(file):

    tables = camelot.read_pdf(file, strip_text='\n', line_scale=40, pages='all', copy_text=['h'])

    if not os.path.exists("results"):
        os.makedirs("results")

    def contains_cid(table):
        for row in table.df.itertuples(index=False):
            for cell in row:
                if "(cid:" in str(cell):
                    return True
        return False
    
    for idx, table in enumerate(tables, 1):
        if contains_cid(table):
            print(f"Font or encoding error in table")
        else:
            output_file = f"results/{os.path.splitext(file)[0]}_table_{idx}.csv"
            table.to_csv(output_file, index=False)

file = '4.pdf'

process_tables(file)

Можно определить тип входного pdf файла и от это задействовать один из сценариев

In [31]:
def is_image_based_pdf(file_path):
    try:
        pdf_file = open(file_path, 'rb')
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        
        # Read the first page of the PDF
        first_page = pdf_reader.pages[0]
        
        # Check if the page contains any text
        return len(first_page.extract_text().strip()) == 0
    
    except Exception as e:
        print(f"Error: {e}")
        return False

# Пример использования
pdf_file = "6_part.pdf"
is_image_based = is_image_based_pdf(pdf_file)

# if is_image_based:
#     print("PDF-документ является image-based")
# else:
#     print("PDF-документ является readable")

PDF-документ является image-based


Для более точного нахождения таблицы на странице будет спользовавться дообученная сеть Mask RCNN

In [23]:
def pdf_to_array(file, resolution=300):
    images = convert_from_path(file, dpi=resolution, poppler_path=r'C:\Program Files\poppler-23.07.0\Library\bin')
    image_arrays = [np.array(image) for image in images]
    return image_arrays

In [26]:
res = pdf_to_array(file='6.pdf')

# for i, image_array in enumerate(res):
#     print(image_array.shape)
#     plt.imshow(cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB))
#     plt.title(f"Page {i+1}")
#     plt.axis('off')
#     plt.show()